In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
# from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

sc = StandardScaler() # Maybe we want to bin continuos data like budget

In [2]:
df = pd.read_excel('../Data/ProcessedData/TMDB_processed.xlsx')
df = df.drop(['title'], axis=1)

In [3]:
features = df.dtypes[(df.columns != 'revenue')].index # Grab all features except that which we are trying to predict

In [ ]:
cv_estimator = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 4, min_child_weight= 5,
                         subsample = 0.8, objective='reg:squarederror')
X_train, X_test, y_train, y_test = train_test_split(df[features], df['revenue'], test_size=0.2, random_state=42)
cv_estimator.fit(X_train, y_train)
cv_selector = RFECV(cv_estimator, cv=5, step=1, scoring='accuracy')
cv_selector = cv_selector.fit(X_train, y_train)
rfecv_mask = cv_selector.get_support() # List of 1's and 0's
rfecv_features = []
for boolean, feature in zip(rfecv_mask, X_train.columns):
    if boolean:
        rfecv_features.append(feature) # Grab only features selected by RFECV
print('Optimal # of Features:', cv_selector.n_features_)
print('Best Features:', rfecv_features)
n_features = X_train.shape[1]
plt.figure(figsize=(25, 35))
plt.barh(range(n_features), cv_estimator.feature_importance_, align='center')
plt.yticks(np.arange(n_features), X_train.columns.values)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()